In [ ]:
import panel as pn
pn.extension()

In [ ]:
import numpy as np

In [ ]:
from libertem_ui.display.notebook_tools import notebook_fullwidth, stop_nb
from align_panel import point_registration, fine_adjust

In [ ]:
notebook_fullwidth()

In [ ]:
shape = (600, 600)
static = np.random.random(size=shape)
moving = np.random.random(size=shape)

In [ ]:
layout, getter = point_registration(static, moving)
layout

In [ ]:
stop_nb()

In [ ]:
res = getter()

In [ ]:
layout, fine_getter = fine_adjust(static, moving, res['transform'])
layout